In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Tikhonov regularisation using CGLS and block framework
Few lines intro

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularization parameter and operator in regulariser.

In [ ]:
from ccpi.framework import ImageGeometry, ImageData 
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import BlockDataContainer

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity

from ccpi.astra.operators import AstraProjectorSimple 

import astra.functions

import tomophantom
from tomophantom import TomoP2D

import numpy as np                          
import matplotlib.pyplot as plt

import os

In [ ]:
#set up aquistion geometry
number_pixels_x = 1024
number_projections = 180
angles = np.linspace(0, np.pi, number_projections, dtype=np.float32)
ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)

#set up image geometry
num_voxels_xy = 1024
ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)

In [ ]:
# Load Shepp-Logan phantom 
model = 1
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

#tomophantom takes angular input in degrees
phantom_2D = TomoP2D.Model(model, num_voxels_xy, path_library2D)
phantom_sino = TomoP2D.ModelSino(model, num_voxels_xy, number_pixels_x, angles*180./np.pi, path_library2D)

sinogram = AcquisitionData(phantom_sino)

In [ ]:
#add Poisson noise to the sinogram
data_noisy = astra.functions.add_noise_to_sino(sinogram.as_array(),400)

sinogram_noisy = ag.allocate()
sinogram_noisy.fill(data_noisy)

In [ ]:
#plot the sinograms
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)

ax1.set_title('Simulated aquistion data')
subplot1 = ax1.imshow(sinogram.as_array())
ax1.set_aspect('auto')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)

ax2.set_title('Noisy aquistion Data')
subplot2 = ax2.imshow(sinogram_noisy.as_array())
ax2.set_aspect('auto')
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)
plt.show()

### Reconstruct using CGLS

In [ ]:
#define the projector
device = "gpu"
Aop = AstraProjectorSimple(ig, ag, device)

#### Reconstruct the clean dataset

In [ ]:
#setup CGLS simple
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=Aop, data=sinogram)
#cgls = CGLS(x_init=x_init, operator=Aop, data=sinogram_noisy)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)
subplot1 = ax1.imshow(cgls.get_output().as_array())
#subplot1.clim(0,2)
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - phantom_2D)
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)
plt.show()

#### Reconstruct the noisy dataset

In [ ]:
#setup CGLS simple
x_init = ig.allocate()      
#cgls = CGLS(x_init=x_init, operator=Aop, data=sinogram)
cgls = CGLS(x_init=x_init, operator=Aop, data=sinogram_noisy)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)
subplot1 = ax1.imshow(cgls.get_output().as_array())
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - phantom_2D)
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)
#subplot1.set_clim(-1.5, 1.5)
plt.show()

### Reconstruct the noisy dataset using regularised CGLS

#### Using the identity operator

In [ ]:
# Setup and run the regularised CGLS
alpha = 20
Id = Identity(ig)

# Form Tikhonov as a Block CGLS structure
op_CGLS = BlockOperator( Aop, alpha * Id, shape=(2,1))
block_data = BlockDataContainer(sinogram_noisy, Id.range_geometry().allocate())

In [ ]:
#setup CGLS with the Block Operator and Block DataContainer
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=op_CGLS, data=block_data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)
subplot1 = ax1.imshow(cgls.get_output().as_array())
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - phantom_2D)
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)
plt.show()

#### Using the gradient operator

In [ ]:
# Setup and run the regularised CGLS
alpha = 50
Grad = Gradient(ig)

# Form Tikhonov as a Block CGLS structure
op_CGLS = BlockOperator( Aop, alpha * Grad, shape=(2,1))
block_data = BlockDataContainer(sinogram_noisy, Grad.range_geometry().allocate())

In [ ]:
#setup CGLS with the block operator and block data
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=op_CGLS, data=block_data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.5)
subplot1 = ax1.imshow(cgls.get_output().as_array())
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - phantom_2D)
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)
plt.show()

# The gradient as a block operator